In [1]:
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, events_methods

In [2]:
base = Path("/home/julienb/Documents/database_scripts/database_scripts_test/fiberevent_files/Rat/Rat101_0729_opto_01/")
event_path = helper.singleglob(base, "Events.csv")
info_path = helper.singleglob(base, "*.yaml", search_upward_limit=Path("/home/julienb/Documents/database_scripts/database_scripts_test/fiberevent_files/"))
res_events_path = base/"events.tsv"
info_path

PosixPath('/home/julienb/Documents/database_scripts/database_scripts_test/fiberevent_files/Rat/rat_model.yaml')

In [3]:
event_df = pd.read_csv(event_path, sep=",").rename(columns=dict(Name="channel_name"))
event_df.insert(0, "t", event_df.pop("TimeStamp")/1000)
event_df

,t,channel_name,State
0,0.962466,Input1,0
1,1.058541,Input1,1
2,12.007255,Input2,0
3,12.103487,Input2,1
4,32.003323,Input4,0
...,...,...,...
355,1172.927029,Input2,0
356,1172.974901,Input1,1
357,1173.102559,Input3,0
358,1173.119513,Input2,1


In [4]:
info = yaml.safe_load(info_path.open("r"))
info

{'processing': [{'duplicate_over': {'channel_name': ['Input(\\d+)']},
   'event_name': '{channel_name}',
   'method': 'input_binary_wave',
   'method_params': {'filter_expr': 'channel_name=="{channel_name}"',
    'state_expr': 'State==0'}}],
 'display': {'rename': {'Input1': 'cue'}}}

In [5]:
event_spec = events_methods.FiberEventProcessing.process_info(event_df["channel_name"].drop_duplicates().to_list(), info["processing"])
pd.DataFrame(list(event_spec.values()))

,event_name,method,method_params
0,Input1,input_binary_wave,"{'filter_expr': 'channel_name==""Input1""', 'sta..."
1,Input2,input_binary_wave,"{'filter_expr': 'channel_name==""Input2""', 'sta..."
2,Input4,input_binary_wave,"{'filter_expr': 'channel_name==""Input4""', 'sta..."
3,Input3,input_binary_wave,"{'filter_expr': 'channel_name==""Input3""', 'sta..."


In [6]:
all=[]
for ev_name, item in event_spec.items():
    ev_dataframe = events_methods.FiberEventProcessing.compute_evdataframe(event_df, item)
    if len(ev_dataframe.index) == 0: continue
    events = events_methods.FiberEventProcessing.call(item["method"],ev_dataframe, item)
    if len(events.index)!=0:
        all.append(events)
all = pd.concat(all).sort_values("t")[["event_name", "t", "duration", "n_segments","metadata", "waveform_changes", "waveform_values"]]
all

,event_name,t,duration,n_segments,metadata,waveform_changes,waveform_values
0,Input1,0.962466,0.096075,1,{},[0],"[0, 1, 0]"
2,Input2,12.007255,0.096232,1,{},[0],"[0, 1, 0]"
4,Input4,32.003323,0.303084,1,{},[0],"[0, 1, 0]"
6,Input1,37.389754,0.096137,1,{},[0],"[0, 1, 0]"
8,Input2,37.629050,0.095993,1,{},[0],"[0, 1, 0]"
...,...,...,...,...,...,...,...
350,Input1,1166.341880,0.145052,1,{},[0],"[0, 1, 0]"
352,Input3,1167.333191,0.144073,1,{},[0],"[0, 1, 0]"
354,Input1,1172.783414,0.191487,1,{},[0],"[0, 1, 0]"
355,Input2,1172.927029,0.192484,1,{},[0],"[0, 1, 0]"


In [7]:
if "display" in info and "rename" in info["display"]:
    all["event_name"] = all["event_name"].map(lambda e: info["display"]["rename"][e] if e in info["display"]["rename"] else e)
json_cols = ["metadata", "waveform_changes", "waveform_values"]
for col in json_cols:
    all[f"{col}_json"] = all[col].apply(lambda d: json.dumps(d))
all.drop(columns=json_cols).to_csv(res_events_path, sep="\t", index=False)
reloaded = pd.read_csv(res_events_path, sep="\t", index_col=False)
for col in reloaded.columns:
    if col.endswith("_json"):
        reloaded[col[:-4]] = reloaded.pop(col).apply(lambda s: json.loads(s) if not pd.isna(s) else None)
reloaded

,event_name,t,duration,n_segments,metadata_,waveform_changes_,waveform_values_
0,cue,0.962466,0.096075,1,{},[0],"[0, 1, 0]"
1,Input2,12.007255,0.096232,1,{},[0],"[0, 1, 0]"
2,Input4,32.003323,0.303084,1,{},[0],"[0, 1, 0]"
3,cue,37.389754,0.096137,1,{},[0],"[0, 1, 0]"
4,Input2,37.629050,0.095993,1,{},[0],"[0, 1, 0]"
...,...,...,...,...,...,...,...
175,cue,1166.341880,0.145052,1,{},[0],"[0, 1, 0]"
176,Input3,1167.333191,0.144073,1,{},[0],"[0, 1, 0]"
177,cue,1172.783414,0.191487,1,{},[0],"[0, 1, 0]"
178,Input2,1172.927029,0.192484,1,{},[0],"[0, 1, 0]"
